In [1]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 
src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset_utils import create_nuscenes_odometry_dataset
from dataset.filters.filter_list import FilterList
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose
import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import shutil

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[(0.267004, 0.004874, 0.329415), (0.277018, 0.050344, 0.375715), (0.282656, 0.100196, 0.42216), (0.28229, 0.145912, 0.46151), (0.276194, 0.190074, 0.493001), (0.265145, 0.232956, 0.516599), (0.252194, 0.269783, 0.531579), (0.235526, 0.309527, 0.542944), (0.21813, 0.347432, 0.550038), (0.201239, 0.38367, 0.554294), (0.185556, 0.41857, 0.556753), (0.171176, 0.45253, 0.557965), (0.159194, 0.482237, 0.558073), (0.14618, 0.515413, 0.556823), (0.133743, 0.548535, 0.553541), (0.123463, 0.581687, 0.547445), (0.119483, 0.614817, 0.537692), (0.128087, 0.647749, 0.523491), (0.150148, 0.676631, 0.506589), (0.19109, 0.708366, 0.482284), (0.24607, 0.73891, 0.452024), (0.311925, 0.767822, 0.415586), (0.386433, 0.794644, 0.372886), (0.468053, 0.818921, 0.323998), (0.545524, 0.838039, 0.275626), (0.636902, 0.856542, 0.21662), (0.730889,

Here we define the dataset depending on nuscenes sequence!

In [2]:
DATASET_PATH = '/media/cedric/Datasets1/nuScenes_mini_v2/nuScenes'
SEQUENCE_NUM = 4

dist_threshold = 5 #moving object filter threshold 

dataset = create_nuscenes_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True, sam_folder_name="SAM_Underseg", dinov2_folder_name="Dinov2",dist_threshold=dist_threshold)

ind_start = 0
ind_end = len(dataset)
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False 
out_folder_ncuts = 'test_data/'
if os.path.exists(out_folder_ncuts):
        shutil.rmtree(out_folder_ncuts)
os.makedirs(out_folder_ncuts)

out_folder = 'pcd_preprocessed_nuscenes/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
Loading nuScenes-panoptic...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
404 panoptic,
Done loading in 0.350 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

In [3]:
if os.path.exists('{out_folder}all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:
        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                major_voxel_size=major_voxel_size,icp=False,
                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                ground_segmentation_method=ground_segmentation_method)


PatchWorkpp::PatchWorkpp() - INITIALIZATION COMPLETE


100%|██████████| 41/41 [00:05<00:00,  7.45it/s]


This cell can be ignored after first run as outputs are stored 

In [4]:
if os.path.exists(f'{out_folder}pcd_ground_minor.pcd') == True:
        pcd_ground_minor, pcd_nonground_minor,\
                all_poses, T_pcd, first_position,labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                        ground_mode=ground_segmentation_method)

        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
        np.savez(f'{out_folder}nuscenes_labels_preprocessed.npz',
                                                instance_nonground= labels['instance_nonground'],
                                                instance_ground= labels['instance_ground'],
                                                seg_ground = labels['seg_ground'],
                                                seg_nonground= labels['seg_nonground']
                                                )


done downsample


In [5]:
pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor.pcd')
pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor.pcd')

nuscenes_labels_orig = {}
with np.load(f'{out_folder}nuscenes_labels_preprocessed.npz') as data :
        nuscenes_labels_orig['instance_ground'] = data['instance_ground']
        nuscenes_labels_orig['instance_nonground'] = data['instance_nonground']
        nuscenes_labels_orig['seg_nonground'] = data['seg_nonground']
        nuscenes_labels_orig['seg_ground'] = data['seg_ground']

        

with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_0.npz') as data:
        all_poses = data['all_poses']
        T_pcd = data['T_pcd']
        first_position = T_pcd[:3, 3]

In [6]:
pcd_new = o3d.geometry.PointCloud()
pcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points))

map_labelled = color_pcd_by_labels(pcd_new,\
                nuscenes_labels_orig['instance_nonground'].reshape(-1,1))

o3d.visualization.draw_geometries([map_labelled])
o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)


i 0 shape (226153,)
i 1 shape (158,)
i 2 shape (521,)
i 3 shape (136,)
i 4 shape (118,)
i 5 shape (5,)
i 6 shape (111,)
i 7 shape (5389,)
i 8 shape (17,)
i 9 shape (99,)
i 10 shape (125,)


True

Now we subsample the poses based on a voxel_size

In [7]:
poses, positions, \
sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start)

100%|██████████| 1/1 [00:00<00:00, 1339.61it/s]


Now we can split the point cloud into chunks based on a tbd chunk_size

In [8]:
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices,indices_ground, center_positions, \
center_ids, chunk_bounds, nuscenes_labels = chunk_and_downsample_point_clouds(pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                            overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=nuscenes_labels_orig)

nuscenes_labels_inlier = copy.deepcopy(nuscenes_labels)

Downsampled from (104668, 3) to (5206, 3) points (non-ground)
Downsampled from (127374, 3) to (6832, 3) points (ground)


In [9]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    
    background_color = np.array([0,0,0])


    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 : 
            pcd_colors[idcs] = background_color
        else : 
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
        
        #if labels[i] != (-1):
        #    pcd_colored.colors[i] = np.array(colors[labels[i]]) / 255
    pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/ 255)
    return pcd_colored

In [15]:
alpha = 0.0
theta = 0.0
beta = 0.0
gamma = 0.1
proximity_threshold = 1.0
colors = generate_random_colors_map(600)
T = 0.05
cams = ["CAM_FRONT", "CAM_FRONT_LEFT", "CAM_FRONT_RIGHT"]
cam_ids = [0]

#out_dbscan = 'out_dbscan/'
#if os.path.exists(out_dbscan) == True : 
#        shutil.rmtree(out_dbscan)
        
#os.makedirs(out_dbscan)

out_nuscenes = 'out_nuscenes/'
if os.path.exists(out_nuscenes) == True : 
        shutil.rmtree(out_nuscenes)

os.makedirs(out_nuscenes)
        
out_nuscenes_instance = 'out_nuscenes_instance/'
if os.path.exists(out_nuscenes_instance) == True : 
        shutil.rmtree(out_nuscenes_instance)
os.makedirs(out_nuscenes_instance)

lower_limit = 0 ##use this for experiments to run limit chunks numberss
upper_limit = 5 ##use this for experiments to run limit chunks numberss

instances = np.hstack((nuscenes_labels_orig['instance_nonground'],nuscenes_labels_orig['instance_ground']))

patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
out_data = []
print(len(center_ids))
for sequence in range(len(center_ids))[lower_limit:upper_limit]:
        if NCUT_ground == False : 
                
                merged_chunk, file_name, pcd_chunk, pcd_chunk_ground, inliers, inliers_ground = ncuts_chunk(dataset, indices, pcd_nonground_chunks, 
                        pcd_ground_chunks, pcd_nonground_chunks_major_downsampling, pcd_nonground_minor, T_pcd, center_positions, center_ids,
                        positions, first_position, sampled_indices_global,
                        chunk_size=chunk_size, major_voxel_size=major_voxel_size,
                        alpha=alpha, beta=beta, gamma=gamma, theta=theta,
                        proximity_threshold=proximity_threshold, ncuts_threshold=T, cams = cams, cam_ids = cam_ids, out_folder=out_folder_ncuts, 
                        ground_mode=False, sequence=sequence, patchwise_indices=patchwise_indices, adjacent_frames_cam=(4,5), adjacent_frames_tarl=(5,5))

                eval_nuscenes = True

                name = file_name.split('/')[-1]
                o3d.io.write_point_cloud(file_name, pcd_chunk + pcd_chunk_ground , write_ascii=False, compressed=False, print_progress=False)

                if eval_nuscenes == True :
                        seg_ground = nuscenes_labels['ground']['semantic'][sequence][inliers][inliers_ground]
                        inst_ground = nuscenes_labels['ground']['instance'][sequence][inliers][inliers_ground]
                        nuscenes_chunk_instance = color_pcd_by_labels(pcd_chunk,nuscenes_labels['nonground']['instance'][sequence].reshape(-1,),
                                        colors=colors,gt_labels=instances)
                        nuscenes_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,inst_ground.reshape(-1,),
                                        colors=colors,gt_labels=instances)
                        o3d.io.write_point_cloud(out_nuscenes_instance + name, nuscenes_chunk_instance + nuscenes_chunk_instance_ground, write_ascii=False, compressed=False, print_progress=False)

                
        else : 
                obstacle_out,file_name = ncuts_chunk(dataset,indices,pcd_nonground_chunks,pcd_ground_chunks,
                        pcd_nonground_chunks_major_downsampling,
                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,sampled_indices_global,
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=True,sequence=sequence,
                        patchwise_indices=patchwise_indices)
                
                
                ground_out,file_name = ncuts_chunk(dataset,indices_ground,pcd_ground_chunks,None,
                        pcd_ground_chunks_major_downsampling,
                        pcd_ground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,sampled_indices_global,
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=True,sequence=sequence,
                        patchwise_indices=patchwise_indices)

                o3d.io.write_point_cloud(file_name, obstacle_out + ground_out, write_ascii=False, compressed=False, print_progress=False)

                print("Pointcloud written to file")
        

1
Start of sequence 0
5206 points in downsampled chunk (major)


KeyboardInterrupt: 

In [11]:
def get_merge_pcds(out_folder_ncuts):
        point_clouds = []

        # List all files in the folder
        files = os.listdir(out_folder_ncuts)
        files.sort()

        # Filter files with a .pcd extension
        pcd_files = [file for file in files if file.endswith(".pcd")]
        print(pcd_files)
        # Load each point cloud and append to the list
        for pcd_file in pcd_files:
                file_path = os.path.join(out_folder_ncuts, pcd_file)
                point_cloud = o3d.io.read_point_cloud(file_path)
                point_clouds.append(point_cloud)
        return point_clouds




def merge_unite_gt(chunks):
    last_chunk = chunks[0] 
    merge = o3d.geometry.PointCloud()
    merge += last_chunk

    for new_chunk in chunks[1:]:
        merge += new_chunk
    
    merge.remove_duplicated_points()
    return merge 

In [12]:
#out_dbscan = 'out_dbscan/'
out_nuscenes = 'out_nuscenes/'
out_nuscenes_instance = 'out_nuscenes_instance/'


point_clouds = get_merge_pcds(out_folder_ncuts)
merge = merge_chunks_unite_instances(point_clouds)


#Merge parts of map
if eval_nuscenes == True : 
    point_clouds_nuscenes_instances = get_merge_pcds(out_nuscenes_instance)
    merge_nuscenes_instance = merge_unite_gt(point_clouds_nuscenes_instances)
    o3d.io.write_point_cloud(out_folder + "merge_part_nuscenes_instance.pcd", merge_nuscenes_instance, write_ascii=False, compressed=False, print_progress=False)
else:
    merge_nuscenes_instance = o3d.io.read_point_cloud('/Users/laurenzheidrich/Documents/Studium/Hiwi_TUM.nosync/Programming/unsup_3d_instances/point-to-pixel-mapping/pcd_preprocessed_nuscenes/merge_part_nuscenes_instance.pcd')



o3d.io.write_point_cloud(out_folder + "merge_part.pcd", merge, write_ascii=False, compressed=False, print_progress=False)


['000011.pcd']
['000011.pcd']


True

In [13]:

unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)
#unique_colors, labels_dbscan = np.unique(np.asarray(merge_dbscan.colors), axis=0, return_inverse=True)
unique_colors, labels_nuscenes = np.unique(np.asarray(merge_nuscenes_instance.colors),axis=0, return_inverse=True)

def intersect(pred_indices, gt_indices):
        intersection = np.intersect1d(pred_indices, gt_indices)
        return intersection.size / pred_indices.shape[0]


def remove_semantics(labels,preds):
        gt_idcs = np.where(labels == 0)[0]
        new_ncuts_labels = preds.copy()
        for i in np.unique(preds):
                pred_idcs = np.where(preds == i)[0]
                cur_intersect = intersect(pred_idcs,gt_idcs)
                if cur_intersect > 0.8:
                        new_ncuts_labels[pred_idcs] = 0
        return new_ncuts_labels

new_ncuts_labels = remove_semantics(labels_nuscenes,labels_ncuts)


metrics_ncuts = Metrics(name='ncuts')
metrics_ncuts.min_points = 50
metrics_ncuts.update_stats(new_ncuts_labels,new_ncuts_labels,labels_nuscenes)

#merge_vis = color_pcd_by_labels(merge,new_ncuts_labels)
#o3d.visualization.draw_geometries([merge_vis])


Metrics for file ncuts
{'panoptic': 0.1954022988505747, 'precision': 0.5, 'recall': 0.25, 'fScore': 0.3333333333333333, 'usr': 0.0, 'osr': 0.0, 'noise': 0.5, 'missed': 0.75, 'mean': 0.5862068965517241}
lstq value :  0.16390875421949833
Average Precision @ 0.25 0.5
Average Precision @ 0.5 0.0625
Average Precision @ 0.55 0.0625
Average Precision @ 0.7 0.0
Average Precision @ 0.75 0.0Average Precision @ 0.6
 0.0
Average Precision @ 0.8 Average Precision @ 0.850.0 
0.0
Average Precision @ 0.9 0.0
Average Precision @ 0.65Average Precision @ 0.95  0.0
0.0
AP @ 0.25 50.0
AP @ 0.5 6.25
AP @ [0.5:0.95] 1.25


In [14]:
merge_vis = color_pcd_by_labels(merge,new_ncuts_labels)
o3d.visualization.draw_geometries([merge_vis])
o3d.visualization.draw_geometries([merge_nuscenes_instance])